In [1]:
import csv
import numpy as np
import tensorflow as tf
import sys

In [2]:
NUMBER_OF_HEROES = 114
DATASET_SIZE = 500000
TEST_RATIO = 0.25

In [3]:
def index_heroes(heroes):
    """ Converts a list of heroes into list of 0s and 1s

    heroes -- list of heroes to be converted
    """

    heroes_indexed = np.zeros(2 * NUMBER_OF_HEROES)

    for i in range(10):
        if i < 5:
            heroes_indexed[int(heroes[i]) - 1] = 1.0
        else:
            heroes_indexed[int(heroes[i]) + 113] = 1.0

    return heroes_indexed

In [4]:
input_file = open('complete_augmented.csv', 'rt')

In [5]:
csv_reader = csv.reader(input_file, delimiter=',')
data_list = list(csv_reader)

In [6]:
train_count = int(DATASET_SIZE * (1 - TEST_RATIO))
test_count = len(data_list) - train_count

x_train = np.zeros((train_count, 2 * NUMBER_OF_HEROES))
y_train = np.zeros((train_count, 2))

x_test = np.zeros((test_count, 2 * NUMBER_OF_HEROES))
y_test = np.zeros((test_count, 2))

# print x_train.shape

In [7]:
for i in range(train_count):
    heroes_indexed = index_heroes(data_list[i][3:13])
    x_train[i] = heroes_indexed
    y_train[i][0] = float(data_list[i][1])
    y_train[i][1] = float(data_list[i][2])
    

for j in range(test_count):
    heroes_indexed = index_heroes(data_list[train_count + j][3:13])
    x_test[j] = heroes_indexed
    y_test[j][0] = float(data_list[train_count + j][1])
    y_test[j][1] = float(data_list[train_count + j][2])


In [8]:
learning_rate = 0.001
training_epochs = 10
batch_size = 100
display_step = 1

In [17]:
n_hidden_1 = 20 # 1st layer number of features
n_input = 2 * NUMBER_OF_HEROES # MNIST data input (img shape: 28*28)
n_classes = 2 # MNIST total classes (0-9 digits)

x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [18]:
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer = tf.nn.relu(layer)

    out_layer = tf.matmul(layer, weights['out']) + biases['out']
    return out_layer

In [19]:
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

In [21]:
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(x_train.shape[0] / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x = x_train[i * batch_size:(i + 1) * batch_size, :]
            batch_y = y_train[i * batch_size:(i + 1) * batch_size, :]
            
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost)
    print "Optimization Finished!"

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({x: x_test, y: y_test})

Epoch: 0001 cost= 1.242029010
Epoch: 0002 cost= 0.673978383
Epoch: 0003 cost= 0.666867588
Epoch: 0004 cost= 0.665596595
Epoch: 0005 cost= 0.665015954
Epoch: 0006 cost= 0.664602598
Epoch: 0007 cost= 0.664230105
Epoch: 0008 cost= 0.663869582
Epoch: 0009 cost= 0.663510806
Epoch: 0010 cost= 0.663149284
Optimization Finished!
Accuracy: 0.595128
